<a href="https://colab.research.google.com/github/ISaySalmonYouSayYes/LLM_mit_LangChain/blob/main/LLM_Agent_Simulate_exp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **0. Before you start**
- The document is [Here](https://github.com/ISaySalmonYouSayYes/LLM_mit_LangChain/blob/main/simulate_exp.md)  
- Contents is always your best friend:P

In [ ]:
import os

try:
    from google.colab import drive, userdata
    COLAB = True
    print("Note: using Google CoLab")
except:
    print("Note: not using Google CoLab")
    COLAB = False

# OpenAI Secrets
if COLAB:
    os.environ["OPENAI_API_KEY"] = userdata.get('openAI_Key')

# Install needed libraries in CoLab
if COLAB:
    !pip install langchain langchain_openai
    !pip install langgraph

# **1. GPT-related method**  
- Using GPT4.0-mini mit Langchain(A library uniforms the syntax of all LLMs)
- Generating Persona(Characters) by LLM
  - Few-shot learning
  - Work perfectly with this prompt

In [2]:
import time
import random

from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain_core.prompts.chat import PromptTemplate

def temp_sleep(seconds=0.1):
  """
  Simulate the realistic delay
  """
  time.sleep(seconds)

def GPT4_request(prompt:str):
  """
  Given a prompt, make a request to OpenAI server and returns the response.
  ARGS:
    prompt: a str prompt
  RETURNS:
    a str of GPT-4's response.
  """
  MODEL = 'gpt-4o-mini'
  messages = [
    SystemMessage(content="user"),
    HumanMessage(content= prompt),
    ]

  llm = ChatOpenAI(
    model=MODEL,
    temperature= 0.0,
    n= 1,
    max_tokens= 256)

  return llm.invoke(messages).content

def GPT4_generate_persona(name:str):
  MODEL = 'gpt-4o-mini'
  prompt = f"""
  Please creating information for {name} following the persona template. Don't add extra words.

  ----------------------------------------
  Template:
      name (str): The name of the persona.
      age (int): Age of the persona.
      gender (str): Gender of the persona.
      status (str): Occupation or current status.
      hobbies (list): List of hobbies the persona enjoys.
      wealth (int): Wealth level (0 to 1000).
      favorite_food (str): Persona's favorite food.
      nemesis (str or None): Persona's nemesis, if any.
      quirky_trait (str or None): A quirky personality trait.
  ----------------------------------------
  Example:
        name="Alex",
        age=29,
        gender="Male",
        status="Freelance Illustrator",
        hobbies=["painting", "cycling", "reading sci-fi"],
        wealth="550",
        favorite_food="Sushi", "Ramen",
        nemesis="Karen from accounting",
        quirky_trait="always wearing mismatched socks"

        name="Jill",
        age=27,
        gender="Female",
        status="Software Developer",
        hobbies=["hiking", "playing video games", "cooking"],
        wealth="600",
        favorite_food="Tacos",
        nemesis="The bug in her code",
        quirky_trait="collects vintage keychains"

  """


  messages = [
    SystemMessage(content="You're a system creating a simulated person"),
    HumanMessage(content= prompt),
    ]

  llm = ChatOpenAI(
    model=MODEL,
    temperature= 0.3,
    n= 1,
    max_tokens= 256)

  return llm.invoke(messages).content


#Debug ---- remove content in return line to see metadata
# print(GPT4_request("Hiii, what is your name?"))

# **2. Persona.py**
- Create an object Persona documenting the personal info
- Generating Persona(Characters) by LLM
  - Few-shot learning
  - Work perfectly with this prompt

In [3]:
class Persona:
    def __init__(self, name, age, gender, status, hobbies, wealth, favorite_food="Pizza", nemesis=None, quirky_trait=None):
        """
        Initializes a Persona instance with a mix of essential and fun attributes.

        Args:
            name (str): The name of the persona.
            age (int): Age of the persona.
            gender (str): Gender of the persona.
            status (str): Occupation or current status.
            hobbies (list): List of hobbies the persona enjoys.
            wealth (int): Wealth level (0 to 1000).
            favorite_food (str): Persona's favorite food (default is "Pizza").
            nemesis (str or None): Persona's nemesis, if any.
            quirky_trait (str or None): A quirky personality trait.
        """
        self.name = name
        self.age = age
        self.gender = gender
        self.status = status
        self.hobbies = hobbies
        self.wealth = wealth  # Clamped between 0 and 1000
        self.favorite_food = favorite_food
        self.nemesis = nemesis
        self.quirky_trait = quirky_trait

    def generate_persona_from_gpt(name:str):
      gpt_response = GPT4_generate_persona(name)
      persona_args = eval(f"dict({gpt_response})")
      return Persona(**persona_args)

    def display_info(self):
        """Displays basic information about the persona."""
        print(f"Name: {self.name}")
        print(f"Age: {self.age}")
        print(f"Gender: {self.gender}")
        print(f"Status: {self.status}")
        print(f"Hobbies: {', '.join(self.hobbies)}")
        print(f"Wealth: {self.wealth}")
        print(f"Favorite Food: {self.favorite_food}")
        print(f"Nemesis: {self.nemesis}")
        print(f"Quirky Trait: {self.quirky_trait}")

    def info_to_dict(self):
      """Returns a dictionary containing the persona's attributes."""
      return {
          "name": self.name,
          "age": self.age,
          "gender": self.gender,
          "status": self.status,
          "hobbies": self.hobbies,
          "wealth": self.wealth,
          "favorite_food": self.favorite_food,
          "nemesis": self.nemesis,
          "quirky_trait": self.quirky_trait,
      }

    def info_to_string_story(self):
      return f'I am {self.name}. My age is {self.age}. My gender is {self.gender}. My status is {self.status}. My hobbies are {self.hobbies}.\
      My wealth is {self.wealth}. My Favorite food is {self.favorite_food}. My nemesis is {self.nemesis}. My Quirky Trait is {self.quirky_trait}'

    def update_info(self, **kwargs):
      """
      Updates the persona's attributes with new values provided as keyword arguments.

      Args:
          **kwargs: Attribute names as keys and their new values as values.
                    Supported attributes: name, age, gender, status, hobbies,
                    wealth, favorite_food, nemesis, quirky_trait.
      """
      for key, value in kwargs.items():
          if hasattr(self, key):
              setattr(self, key, value)
              print(f"Updated {key} to {value}")
          else:
              print(f"Attribute {key} not found in Persona.")

# **3. Talk.py**

In [4]:
import re
from langchain.chains import ConversationChain
from langchain.memory import ConversationSummaryBufferMemory
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain_core.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)
from IPython.display import display_markdown

class Talk:
  def begin_conversation(persona_name, opposite_persona_name, persona_info, opposite_persona_info):
      MODEL = 'gpt-4o-mini'

      system_message = (
      f"You are {persona_name}. Here is your background:\n"
      f"{persona_info}\n\n"
      f"You are speaking with {opposite_persona_name}, whose background is:\n"
      f"{opposite_persona_info}\n\n"
      "Guidelines:\n"
      "1. Format answers with markdown.\n"
      "2. Answer based on the background information provided.\n"
      f"3. Repeat your name before answering (e.g., \"{persona_name}: ...\").\n"
      "4. Stick to the original topic.\n"
      "5. Avoid repeating yourself.\n"
      )


      # Create system and human message templates
      system_message_template = SystemMessagePromptTemplate.from_template(
          system_message
      )


      human_message_template = HumanMessagePromptTemplate.from_template(
          """
          Conversation: {history}
          Human: {input}
          """
      )

      # Combine into a ChatPromptTemplate
      prompt_template = ChatPromptTemplate.from_messages(
          [system_message_template, human_message_template]
      )

      # Initialize the OpenAI LLM
      llm = ChatOpenAI(
          model=MODEL,
          temperature=0.0,
          n=1,
          max_tokens= 100
      )

      # Initialize memory with auto-summarization
      memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=300)

      # Build a conversation chain
      conversation = ConversationChain(
          prompt=prompt_template,
          llm=llm,
          memory=memory,
          verbose=False,
      )

      return conversation


  def chat(conversation, prompt):
      """
      Simulate a single conversational turn.

      Args:
          conversation: The ConversationChain object.
          prompt (str): The message from the human.
          name (str): The name of the persona responding.

      Returns:
          str: The AI's response.
      """
      # Pass the prompt to the conversation chain
      response = conversation.invoke({"input": prompt})
      response_text = response['response']

      # Display the response in markdown
      display_markdown(f"{response_text}", raw=True)
      return response_text

  def summarize_after_chat(conversation, persona_name, opposite_persona_name, persona_info, opposite_persona_info):
      """
      Summarize the conversation after the last turn.

      Args:
          conversation: The ConversationChain object.
      """
      SYSTEM_PROMPT = (
      f"You are {persona_name} and the following is your background:\n"
      f"Your background:\n{persona_info}\n"
      f"YYou are in a conversation with {opposite_persona_name}, who has the following background information:\n"
      f"{opposite_persona_name}'s Background:\n{opposite_persona_info}\n\n"
      f"During the conversation, identify any changes to your own background information (not {opposite_persona_name}'s). \
      Clearly describe the changes as they apply to your persona only.\n"
      "Guidelines:\n"
      "1. Format answers with markdown.\n"
      "2. Ignore human and AI in the conversation\n"
      "3. If your response includes information that would result in\
       a change to your background({name, age, gender, status, hobbies, wealth, favorite_food, nemesis, quirky_trait}),\
        please repeat the change in the following format: <changeCategoryBefore></changeCategoryBefore> and <changeCategoryAfter></changeCategoryAfter>\n"
      "4. Reason your change, if it doesn't make sense, reconsider again"
      "Example:\nBelly borrow 50 from Cindy. Belly's wealth is growing from <wealthBefore>300</wealthBefore> to <wealthAfter>350</wealthAfter>\n\
      Cindy lend 50 to Cindy. Cindy's wealth is decreasing from <wealthBefore>1000</wealthBefore> to <wealthAfter>950</wealthAfter>"
      )

      HUMAN_PROMPT = conversation.memory.buffer
      # print(HUMAN_PROMPT)

      MODEL = 'gpt-4o-mini'
      messages = [
        SystemMessage(content = SYSTEM_PROMPT),
        HumanMessage(content = HUMAN_PROMPT),
        ]

      llm = ChatOpenAI(
        model=MODEL,
        temperature= 0.0,
        n= 1,
        max_tokens= 300)

      return llm.invoke(messages).content

  # def summarize_for_diary(conversation, persona_name, opposite_persona_name, persona_info, opposite_persona_info):


  def retrieve_and_upgrade_persona(persona, summary):
      """
      Extracts 'after' updates for the fields: name, age, gender, status, hobbies,
      wealth, favorite_food, nemesis, quirky_trait from a summary and updates the Persona object.

      Args:
          persona (Persona): The Persona object to update.
          summary (str): The string containing updates with after values.
      """
      print(summary)
      # Define the attributes and their regex patterns for 'after'
      attributes = ["name", "age", "gender", "status", "hobbies", "wealth", "favorite_food", "nemesis", "quirky_trait"]
      updates = {}

      for attribute in attributes:
          after_pattern = rf"<{attribute}After>(.*?)</{attribute}After>"

          after_match = re.search(after_pattern, summary)

          if after_match:
              value = after_match.group(1)
              # Parse hobbies if the attribute is 'hobbies'
              if attribute == "hobbies":
                  # Convert the hobbies string to a Python list
                  value = [hobby.strip() for hobby in value.split(',') if hobby.strip()]
              updates[attribute] = value

      # Update the persona object using the update_info method
      persona.update_info(**updates)

  def persona_to_persona_chat(persona_1, persona_2, persona_2_intro, rounds=5):
      """
      Simulates a conversation between two personas using ConversationChain.

      Args:
          persona_1_name (str): The name of the first persona.
          persona_2_name (str): The name of the second persona.
          persona_2_intro (str): The introductory message from Persona 2.
          rounds (int): Number of conversational exchanges.
      """
      persona_1_name = persona_1.name
      persona_2_name = persona_2.name
      persona_1_info = persona_1.info_to_string_story()
      persona_2_info = persona_2.info_to_string_story()



      # Initialize two separate conversations for each persona
      persona_1_conversation = Talk.begin_conversation(persona_1_name, persona_2_name, persona_1_info, persona_2_info)
      persona_2_conversation = Talk.begin_conversation(persona_2_name, persona_1_name, persona_2_info, persona_1_info)

      persona_2_intro = f"{persona_2_name}: {persona_2_intro}."

      # Start the conversation
      print("=== Conversation Start ===")
      persona_1_response = Talk.chat(persona_1_conversation, persona_2_intro)

      # persona_1_response = f"{persona_2_name}: {persona_2_intro}\n {persona_1_response}"
      persona_1_response = f"{persona_2_name}: {persona_2_intro}\n {persona_1_name}: {persona_1_response}."
      for i in range(rounds):
          persona_2_response = Talk.chat(persona_2_conversation, persona_1_response)
          persona_1_response = Talk.chat(persona_1_conversation, persona_2_response)
      print("=== Conversation End ===")
      Talk.retrieve_and_upgrade_persona(persona_1, Talk.summarize_after_chat(persona_1_conversation, persona_1_name, persona_2_name, persona_1_info, persona_2_info))
      Talk.retrieve_and_upgrade_persona(persona_2, Talk.summarize_after_chat(persona_2_conversation, persona_2_name, persona_1_name, persona_2_info, persona_1_info))


      # print(f"Summary: {Talk.summarize_after_chat(persona_1_conversation, persona_1_name, persona_2_name, persona_1_info, persona_2_info)}")
      # print(f"Summary: {Talk.summarize_after_chat(persona_2_conversation, persona_2_name, persona_1_name, persona_2_info, persona_1_info)}")


# **4. Run here!**

In [5]:
persona_lea = Persona.generate_persona_from_gpt("Lea")
persona_alex = Persona(
    name="Alex",
    age=29,
    gender="Male",
    status="Teacher",
    hobbies=["painting", "cycling", "reading sci-fi"],
    wealth="550",
    favorite_food="Sushi",
    nemesis="Karen from accounting",
    quirky_trait="always wearing mismatched socks"
)
Persona.display_info(persona_lea)

Name: Lea
Age: 32
Gender: Female
Status: Graphic Designer
Hobbies: photography, yoga, traveling
Wealth: 450
Favorite Food: Pasta
Nemesis: The printer that never works
Quirky Trait: talks to her plants


## **4.1 Case 1: Asking Alex to provide more money than he has**
**Observation:** Alex know he doesn't have that much money.  
**Problem:** The reply contains too much info...need to fix the prompt.

In [ ]:
# Define persona information
persona_1_name = "Alex"
persona_2_name = "Lea"
persona_2_intro = "Can I borrow 1000 from you?"
persona_2_info = persona_lea.info_to_string_story()
persona_1_info = persona_alex.info_to_string_story()

# Simulate their conversation
Talk.persona_to_persona_chat(persona_alex, persona_lea, persona_2_intro, rounds=1)
# persona_to_persona_chat(persona_1_name, persona_2_name, persona_2_intro, rounds=5)

=== Conversation Start ===


Alex: I appreciate you asking, Lea, but I'm not in a position to lend that amount right now. My wealth is a bit limited at 550. Maybe we can brainstorm some other options together?

Lea: I completely understand, Alex. It's tough when finances are tight. Maybe we can think of some creative ways to save or earn a little extra? I could even design some promotional materials for you if you're looking to sell some of your artwork!

Alex: That sounds like a great idea, Lea! I could definitely use some help promoting my artwork. Your graphic design skills would really make a difference. Plus, I love the idea of collaborating creatively. Let's brainstorm some promotional concepts together!

=== Conversation End ===
In this conversation, there are no changes to my background information. My wealth remains at 550, and I continue to be a teacher with my hobbies, favorite food, nemesis, and quirky trait unchanged. 

Therefore, no changes to report.
No changes to my background information occurred during this conversation.


## **4.2.1 Borrow reasonable money from Alex**  
**Observation:**
1. Alex know he can lend the money to Lea.
2. Summary system know this would change both people's wealth.
3. Upgrade info retrieval method can identify the change.
**Problem:** Summary system made some mistake at some point.

In [ ]:
# Define persona information
persona_1_name = "Alex"
persona_2_name = "Lea"
persona_2_intro = "Can I borrow 20 from you? I'll give you back in a month"
persona_2_info = persona_lea.info_to_string_story()
persona_1_info = persona_alex.info_to_string_story()

# Simulate their conversation
Talk.persona_to_persona_chat(persona_alex, persona_lea, persona_2_intro, rounds=2)

=== Conversation Start ===


Alex: Sure, Lea! I can lend you the 20. Just let me know when you need it back.

Lea: Thanks, Alex! I really appreciate it. I promise to pay you back in a month. By the way, have you been painting anything interesting lately?

Alex: You're welcome, Lea! I'm glad to help. As for painting, I've been working on a piece inspired by sci-fi themes. It's a bit abstract, but I'm really enjoying the process. How about you? Have you been working on any photography projects?

Lea: That sounds intriguing, Alex! I love the idea of combining sci-fi with abstract art. As for me, I've been focusing on capturing some travel moments lately. I recently went to a beautiful place and took some stunning shots. Do you have a favorite sci-fi book that inspires your painting?

Alex: Thanks, Lea! I'm glad you find it intriguing. One of my favorite sci-fi books is "Dune" by Frank Herbert. The world-building and themes really inspire my creativity. What about you? Do you have a favorite travel destination that has influenced your photography?

=== Conversation End ===
During the conversation, I lent Lea 20, which affects my wealth. 

My wealth is decreasing from <wealthBefore>550</wealthBefore> to <wealthAfter>530</wealthAfter>. 

This change reflects the amount I lent to Lea, and I will expect to receive it back in a month.
Updated wealth to 530
Lea's wealth is decreasing from <wealthBefore>450</wealthBefore> to <wealthAfter>430</wealthAfter> due to borrowing 20 from Alex.
Updated wealth to 430


## **4.2.2 Borrow reasonable money from Alex(longer conversation)**  
**Observation:**
1. Alex invite Lea to do something! Lea say yes!
**Problem:** Diary system is not ready to capture the event LoL

In [7]:
# Define persona information
persona_1_name = "Alex"
persona_2_name = "Lea"
persona_2_intro = "Can I borrow 20 from you? I'll give you back in a month"
persona_2_info = persona_lea.info_to_string_story()
persona_1_info = persona_alex.info_to_string_story()

# Simulate their conversation
Talk.persona_to_persona_chat(persona_alex, persona_lea, persona_2_intro, rounds=10)

=== Conversation Start ===


Alex: I appreciate the ask, Lea, but I'm not in a position to lend money right now. My wealth is a bit tight at 550. Maybe we can brainstorm some other ways to help you out?

Lea: I totally understand, Alex! It's tough when finances are tight. Maybe we can think of some creative ways to save money together or even find some fun, low-cost activities to do. What do you think?

Alex: That sounds like a great idea, Lea! We could definitely brainstorm some fun, low-cost activities. How about a painting and photography day in the park? I can bring my art supplies, and you can capture some great shots. Plus, we can enjoy the outdoors while saving some money! What do you think?

Lea: I love that idea, Alex! A painting and photography day in the park sounds like a perfect way to combine our hobbies. I can bring my camera and maybe even some snacks to enjoy while we work on our art. Plus, talking to my plants beforehand might give me some extra inspiration! When are you thinking of going?

...

Lea: Absolutely, Alex! I can already envision the beautiful moments we’ll capture and create together. It’s going to be a fantastic day filled with inspiration and laughter. I’m ready to embrace the creativity and have a blast! See you at 10 AM!

Alex: I love your enthusiasm, Lea! It's great to hear that you're already envisioning the day. I can’t wait to see how your photography captures the essence of my paintings. It’s going to be a wonderful blend of creativity and joy. Let’s make the most of it! See you at 10 AM!

Lea: Thank you, Alex! Your excitement is contagious! I’m really looking forward to capturing the essence of your paintings through my lens. It’s going to be such a unique experience, and I can already tell we’ll have a lot of fun. Let’s make it a day to remember! See you at 10 AM!

Alex: Thank you, Lea! I'm thrilled that my excitement resonates with you! I believe this collaboration will be a unique experience, and I can't wait to see how your lens interprets my art. Let's definitely make it a memorable day filled with creativity and laughter. See you at 10 AM!

=== Conversation End ===
During the conversation, there are no changes to my background information. I maintained my status as a teacher, my hobbies, wealth, favorite food, nemesis, and quirky trait remained the same. The interaction focused on planning a creative day with Lea without any alterations to my persona.
During the conversation, there are no changes to my background information. I remain:

- Name: Lea
- Age: 32
- Gender: Female
- Status: Graphic Designer
- Hobbies: ['photography', 'yoga', 'traveling']
- Wealth: 450
- Favorite food: Pasta
- Nemesis: The printer that never works
- Quirky Trait: Talks to her plants

The interaction with Alex was focused on planning a creative day together, but it did not result in any changes to my background.


In [9]:
Persona.display_info(persona_alex)
Persona.display_info(persona_lea)

Name: Alex
Age: 29
Gender: Male
Status: Teacher
Hobbies: painting, cycling, reading sci-fi
Wealth: 550
Favorite Food: Sushi
Nemesis: Karen from accounting
Quirky Trait: always wearing mismatched socks
Name: Lea
Age: 32
Gender: Female
Status: Graphic Designer
Hobbies: photography, yoga, traveling
Wealth: 450
Favorite Food: Pasta
Nemesis: The printer that never works
Quirky Trait: talks to her plants


# **5. Diary.py**

In [11]:
class Diary:
    def __init__(self):
        """
        Initialize the Diary class with an empty dictionary to store events.
        """
        self.events = {}
        self.to_do_list = {}

    def add_event(self, date, description):
        """
        Add a new event to the diary.

        :param date: The date of the event (string format: 'YYYY-MM-DD').
        :param description: A description of the event (string).
        """
        if date in self.events:
            self.events[date].append(description)
        else:
            self.events[date] = [description]
        print(f"Event added for {date}: {description}")

    def view_events(self, date):
        """
        View all events for a specific date.

        :param date: The date to view events (string format: 'YYYY-MM-DD').
        :return: A list of events for the specified date or a message if no events exist.
        """
        if date in self.events:
            return self.events[date]
        else:
            return f"No events found for {date}."

    def delete_event(self, date, description):
        """
        Delete a specific event from a specific date.

        :param date: The date of the event to delete (string format: 'YYYY-MM-DD').
        :param description: The description of the event to delete (string).
        """
        if date in self.events:
            try:
                self.events[date].remove(description)
                print(f"Event removed for {date}: {description}")
                if not self.events[date]:  # If the list for that date is empty, remove the key
                    del self.events[date]
            except ValueError:
                print(f"Event not found for {date}: {description}")
        else:
            print(f"No events found for {date}.")

    def list_all_events(self):
        """
        List all events in the diary.

        :return: A dictionary containing all events or a message if the diary is empty.
        """
        if self.events:
            return self.events
        else:
            return "No events in the diary."

    def add_to_do_item(self, date, description):
        """
        Add a new to-do item to the diary.

        :param date: The date of the to-do item (string format: 'YYYY-MM-DD').
        :param description: A description of the to-do item (string).
        """
        if date in self.to_do_list:
            self.to_do_list[date].append(description)
        else:
            self.to_do_list[date] = [description]

    def view_to_do_list(self):
        """
        View all to-do items in the diary.

        :return: A dictionary containing all to-do items or a message if the to-do list is empty.
        """
        if self.to_do_list:
            return self.to_do_list
        else:
            return "No to-do items in the diary."

    def delete_to_do_item(self, date, description):
        if date in self.to_do_list:
            try:
                self.to_do_list[date].remove(description)
                print(f"Event removed for {date}: {description}")
                if not self.to_do_list[date]:  # If the list for that date is empty, remove the key
                    del self.to_do_list[date]
            except ValueError:
                print(f"Event not found for {date}: {description}")
        else:
            print(f"No to_do_list found for {date}.")

# Example usage:
if __name__ == "__main__":
    my_diary = Diary()
    my_diary.add_event("2024-12-18", "Submit thesis")
    my_diary.add_event("2024-12-18", "Celebrate with friends")
    my_diary.add_event("2024-12-19", "Plan holiday trip")

    # print(my_diary.view_events("2024-12-18"))
    # my_diary.delete_event("2024-12-18", "Submit thesis")
    # print(my_diary.view_events("2024-12-18"))
    print(my_diary.list_all_events())



Event added for 2024-12-18: Submit thesis
Event added for 2024-12-18: Celebrate with friends
Event added for 2024-12-19: Plan holiday trip
{'2024-12-18': ['Submit thesis', 'Celebrate with friends'], '2024-12-19': ['Plan holiday trip']}


# **98. What to do next**

1. Retrieve new info from a conversation, and then upgrade the Persona.

# **99. Working History**

- 2024.12.16 ---- Rewrite the prompt in SystemMessage(GPT4_generate_persona)